## Climbing Shoe Website Parser

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners
import pyperclip

In [2]:
# Creating the soup using html.parser

mec_url = "https://www.mec.ca/en/products/climbing/climbing-footwear/climbing-shoes?promotions=clearance"
mec_page = requests.get(mec_url)
mec_soup = BeautifulSoup(mec_page.content, "html.parser")

df = pd.DataFrame(columns=["Shoe","Sale Price","Original Price","Discount","URL","Shipping Cost"])

In [3]:
#Parse specific MEC.ca related content tags
results = mec_soup.find(id="main-content")
mec_sale_elements = results.find_all("li", class_="Hits_searchHitsItem__XaIFr")

In [7]:
#Loop through each sale element for relevant information
for sale_element in mec_sale_elements:
    shoe_name_element = sale_element.find("a", class_="Hit_hitTitle__dl_OY")
    price_element = sale_element.find("span", class_="Hit_hitPrices__IHAvd").find("span")
    og_price_element = sale_element.find("span", class_="Hit_hitPrices__IHAvd").find("span", class_="Hit_hitOldPrice__7i8ik")
    link_element = "mec.ca" + sale_element.find("a")["href"]
    new_row = {"Shoe" : shoe_name_element,
               "Sale Price" : float(price_element.text.strip()[1:]),
               "Original Price" : float(og_price_element.text.strip()[1:]),
               "Discount" : str(round((float(og_price_element.text.strip()[1:])-float(price_element.text.strip()[1:]))/float(og_price_element.text.strip()[1:]),2)*100)+"%",
               "URL" : link_element,
               "Shipping Cost" : 0
              }
    new_row_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_row_df],ignore_index=True)

type_tiny = pyshorteners.Shortener() 
df["URL"] = df["URL"].apply(lambda x: type_tiny.tinyurl.short(x))
df["Post-Tax Price"] = (df["Sale Price"]+df["Shipping Cost"])*1.13

ShorteningErrorException: There was an error on trying to short the url: b'Error'

In [5]:
#Output the table with values sorted
df.sort_values(by="Discount",ascending=False)

,Shoe,Sale Price,Original Price,Discount,URL,Shipping Cost,Post-Tax Price
8,[Black Diamond Zone LV Rock Shoes - Unisex],49.93,129.95,62.0%,https://tinyurl.com/22weqoy5,0,56.4209
5,[Scarpa Mago Rock Shoes - Unisex],139.93,248.95,44.0%,https://tinyurl.com/2445ojlr,0,158.1209
3,[Scarpa Quantic Climb Shoes - Men's],128.94,198.95,35.0%,https://tinyurl.com/2cg8dg52,0,145.7022
6,[Scarpa Quantic Climb Shoes - Women's],128.94,198.95,35.0%,https://tinyurl.com/27p6tqoj,0,145.7022
0,[Scarpa Boostic Rock Shoes - Unisex],186.94,248.95,25.0%,https://tinyurl.com/2yj4urtq,0,211.2422
1,[Scarpa Force V Rock Shoes - Men's],148.94,198.95,25.0%,https://tinyurl.com/23wxwcpx,0,168.3022
2,[Scarpa Force V Rock Shoes - Women's],148.94,198.95,25.0%,https://tinyurl.com/2cn3p3br,0,168.3022
4,[Scarpa Reflex V Rock Shoes - Men's],104.94,138.95,24.0%,https://tinyurl.com/2azqljxa,0,118.5822
7,[Scarpa Reflex V Rock Shoes - Women's],104.94,138.95,24.0%,https://tinyurl.com/247jud8n,0,118.5822


In [6]:
# Potential Future Multiple website Improvement
# url_list = ["https://www.mec.ca/en/products/climbing/climbing-footwear/climbing-shoes?promotions=clearance": 0,
#             "https://www.sail.ca/en/outdoor-gear/climbing/climbing-shoes#/filter:ss_special_price:Yes": 0]

# for i[0] in url_list:
#     tmppage = ""
#     tmppage = requests.get(i)
#     website_name = i.split("www.",)[1].split(".ca")[0] + "_soup"
#     website_name = BeautifulSoup(tmppage.content, "html.parser")